In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import os

import torch
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx
from scipy import sparse
from scipy.linalg import fractional_matrix_power

from utils import *
from models import GNN
from dataset_utils import DataLoader

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.001,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=8e-3,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=64,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.9,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--dataset', default='cora', help='Dataset name.')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='cora', type=None, choices=None, help='Dataset name.', metavar=None)

In [3]:
args = parser.parse_args("")

In [4]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)

In [5]:
dname = args.dataset
dataset = DataLoader(dname)
data = dataset[0]

In [6]:
A_norm, A, X, labels, idx_train, idx_val, idx_test = load_citation_data(data)

In [7]:
G = nx.from_numpy_matrix(A)
feature_dictionary = {}

for i in np.arange(len(labels)):
    feature_dictionary[i] = labels[i]

nx.set_node_attributes(G, feature_dictionary, "attr_name")

In [8]:
sub_graphs = []

for i in np.arange(len(A)):
    s_indexes = []
    for j in np.arange(len(A)):
        s_indexes.append(i)
        if(A[i][j]==1):
            s_indexes.append(j)
    sub_graphs.append(G.subgraph(s_indexes))

subgraph_nodes_list = []

for i in np.arange(len(sub_graphs)):
    subgraph_nodes_list.append(list(sub_graphs[i].nodes))

In [9]:
sub_graphs_adj = []
for index in np.arange(len(sub_graphs)):
    sub_graphs_adj.append(nx.adjacency_matrix(sub_graphs[index]).toarray())

In [10]:
new_adj = torch.zeros(A.shape[0], A.shape[0])

for node in np.arange(len(subgraph_nodes_list)):
    sub_adj = sub_graphs_adj[node]
    for neighbors in np.arange(len(subgraph_nodes_list[node])):
        index = subgraph_nodes_list[node][neighbors]
        count = torch.tensor(0).float()
        if(index==node):
            continue
        else:
            c_neighbors = set(subgraph_nodes_list[node]).intersection(subgraph_nodes_list[index])
            if index in c_neighbors:
                nodes_list = subgraph_nodes_list[node]
                sub_graph_index = nodes_list.index(index)
                c_neighbors_list = list(c_neighbors)
                for i, item1 in enumerate(nodes_list):
                    if(item1 in c_neighbors):
                        for item2 in c_neighbors_list:
                            j = nodes_list.index(item2)
                            count += sub_adj[i][j]

            new_adj[node][index] = count/2
            new_adj[node][index] = new_adj[node][index]/(len(c_neighbors)*(len(c_neighbors)-1))
            new_adj[node][index] = new_adj[node][index] * (len(c_neighbors)**1)

In [11]:
features = torch.FloatTensor(X)
labels = torch.LongTensor(labels)

weight = torch.FloatTensor(new_adj)
weight = weight / weight.sum(1, keepdim=True)

weight = weight + torch.FloatTensor(A)

coeff = weight.sum(1, keepdim=True)
coeff = torch.diag((coeff.T)[0])

weight = weight + coeff

In [12]:
weight = weight.detach().numpy()
weight = np.nan_to_num(weight, nan=0)

row_sum = np.array(np.sum(weight, axis=1))
degree_matrix = np.matrix(np.diag(row_sum+1))

D = fractional_matrix_power(degree_matrix, -0.5)
A_tilde_hat = D.dot(weight).dot(D)

In [13]:
adj = torch.FloatTensor(A_tilde_hat)

In [14]:
# Model and optimizer
model = GNN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

In [15]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately, deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [16]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [17]:
# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.9803 acc_train: 0.1429 loss_val: 1.8961 acc_val: 0.2100 time: 0.4049s
Epoch: 0002 loss_train: 1.9559 acc_train: 0.1500 loss_val: 1.8945 acc_val: 0.2140 time: 0.4199s
Epoch: 0003 loss_train: 1.9697 acc_train: 0.1643 loss_val: 1.8928 acc_val: 0.2140 time: 0.4159s
Epoch: 0004 loss_train: 1.9813 acc_train: 0.1571 loss_val: 1.8911 acc_val: 0.2180 time: 0.4279s
Epoch: 0005 loss_train: 1.9538 acc_train: 0.1571 loss_val: 1.8893 acc_val: 0.2300 time: 0.4129s
Epoch: 0006 loss_train: 1.9510 acc_train: 0.2000 loss_val: 1.8876 acc_val: 0.2420 time: 0.4009s
Epoch: 0007 loss_train: 1.9517 acc_train: 0.1643 loss_val: 1.8859 acc_val: 0.2480 time: 0.4189s
Epoch: 0008 loss_train: 1.9364 acc_train: 0.1929 loss_val: 1.8842 acc_val: 0.2520 time: 0.4209s
Epoch: 0009 loss_train: 1.9380 acc_train: 0.1714 loss_val: 1.8826 acc_val: 0.2600 time: 0.3989s
Epoch: 0010 loss_train: 1.9356 acc_train: 0.1643 loss_val: 1.8811 acc_val: 0.2640 time: 0.4079s
Epoch: 0011 loss_train: 1.9317 acc_train

Epoch: 0087 loss_train: 1.4926 acc_train: 0.7357 loss_val: 1.6541 acc_val: 0.7400 time: 0.4647s
Epoch: 0088 loss_train: 1.4965 acc_train: 0.7143 loss_val: 1.6493 acc_val: 0.7420 time: 0.4417s
Epoch: 0089 loss_train: 1.4303 acc_train: 0.7714 loss_val: 1.6445 acc_val: 0.7440 time: 0.4309s
Epoch: 0090 loss_train: 1.4508 acc_train: 0.7214 loss_val: 1.6396 acc_val: 0.7460 time: 0.4644s
Epoch: 0091 loss_train: 1.4527 acc_train: 0.7714 loss_val: 1.6347 acc_val: 0.7480 time: 0.4257s
Epoch: 0092 loss_train: 1.4583 acc_train: 0.7643 loss_val: 1.6298 acc_val: 0.7500 time: 0.4398s
Epoch: 0093 loss_train: 1.4275 acc_train: 0.7429 loss_val: 1.6248 acc_val: 0.7520 time: 0.4479s
Epoch: 0094 loss_train: 1.4201 acc_train: 0.7429 loss_val: 1.6197 acc_val: 0.7540 time: 0.4649s
Epoch: 0095 loss_train: 1.4193 acc_train: 0.7429 loss_val: 1.6146 acc_val: 0.7540 time: 0.4248s
Epoch: 0096 loss_train: 1.4085 acc_train: 0.7786 loss_val: 1.6094 acc_val: 0.7600 time: 0.4589s
Epoch: 0097 loss_train: 1.4357 acc_train

Epoch: 0173 loss_train: 0.8384 acc_train: 0.8857 loss_val: 1.2138 acc_val: 0.7900 time: 0.4982s
Epoch: 0174 loss_train: 0.9054 acc_train: 0.8571 loss_val: 1.2096 acc_val: 0.7880 time: 0.6110s
Epoch: 0175 loss_train: 0.9198 acc_train: 0.8357 loss_val: 1.2056 acc_val: 0.7880 time: 0.5052s
Epoch: 0176 loss_train: 0.8332 acc_train: 0.8929 loss_val: 1.2016 acc_val: 0.7900 time: 0.5322s
Epoch: 0177 loss_train: 0.8237 acc_train: 0.9214 loss_val: 1.1976 acc_val: 0.7900 time: 0.5139s
Epoch: 0178 loss_train: 0.8544 acc_train: 0.8643 loss_val: 1.1935 acc_val: 0.7900 time: 0.5426s
Epoch: 0179 loss_train: 0.8569 acc_train: 0.8786 loss_val: 1.1895 acc_val: 0.7920 time: 0.5495s
Epoch: 0180 loss_train: 0.7805 acc_train: 0.8857 loss_val: 1.1855 acc_val: 0.7920 time: 0.5550s
Epoch: 0181 loss_train: 0.8470 acc_train: 0.8643 loss_val: 1.1817 acc_val: 0.7920 time: 0.5792s
Epoch: 0182 loss_train: 0.7640 acc_train: 0.9357 loss_val: 1.1779 acc_val: 0.7920 time: 0.5656s
Epoch: 0183 loss_train: 0.7688 acc_train